In [1]:
import sys
sys.path.append('../')
from rules.metric import *

In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import itertools
from sklearn.metrics import precision_recall_fscore_support

In [15]:
raw_data = pd.read_csv('/data/wujipeng/ec/data/han/processed_data.csv')
rule_labels = test_rules(raw_data, range(1,3), strict=False)

In [16]:
rule_labels = np.sign(list(map(sum, rule_labels)))

In [19]:
pickle.dump(rule_labels, open('/data/wujipeng/ec/data/raw_data/rule_labels.pkl', 'wb'))

In [56]:
def sign(x):
    if x > 0.5:
        return 1
    return 0

In [191]:
def stat_rule(model, p=1, r=0.5):
    fp_samples = None
    tpfp = []
    all_gain = []
    for time in range(1, len(os.listdir(os.path.join(eval_root, model))) + 1):
        if not os.path.exists(os.path.join(eval_root, model, model + '.' + str(time),
                         'statistics_final_best.csv')):
            break
        print('Eval time {}'.format(time))
        # 用 pd.read_csv 由于分隔符读取有问题，会出错

        with open(
                os.path.join(eval_root, model, model + '.' + str(time),
                             'statistics_final_best.csv')) as f:
            columns = f.readline().strip().split('\t')
            data = [line.strip().split('\t') for line in f.readlines()]
            stat = pd.DataFrame(
                data=data, columns=columns).rename(
                    {
                        'test_results[0]': 'prob0',
                        'test_results[1]': 'prob1'
                    },
                    axis=1).apply(
                        pd.to_numeric, errors='ignore')
        val_set = pickle.load(
            open('/data/wujipeng/ec/data/static/static.{}/split.pkl'.format(time),
                 'rb'))[1]
        rule_preds = list(
            itertools.chain.from_iterable(
                pd.Series(rule_labels,
                          index=range(1,
                                      len(rule_labels) + 1))[val_set].tolist()))
        stat['rule_preds'] = rule_preds

        stat['rprob1'] = np.add(np.multiply(rule_preds, p - 0.5), 0.5)
        stat['rprob0'] = 1 - stat['rprob1']
        stat['eprob0'] = r * stat['prob0'] + (1 - r) * stat['rprob0']
        stat['eprob1'] = 1 - stat['eprob0']
        stat['ensem_preds'] = stat['eprob1'].apply(sign)
        stat = stat[[
            'SUCC', 'rule_preds', 'test_preds', 'ensem_preds', 'test_labels',
            'prob0', 'prob1', 'rprob0', 'rprob1', 'eprob0', 'eprob1', 'sentence',
            'clause', 'predict', 'real', 'succ', 'keyword', 'content'
        ]]

        mtp = len(stat[(stat['test_preds'] == 1) & (stat['test_labels'] == 1)])
        mfp = len(stat[(stat['test_preds'] == 1) & (stat['test_labels'] == 0)])
        rtp = len(stat[(stat['rule_preds'] == 1) & (stat['test_labels'] == 1)])
        rfp = len(stat[(stat['rule_preds'] == 1) & (stat['test_labels'] == 0)])
        etp = len(stat[(stat['ensem_preds'] == 1) & (stat['test_labels'] == 1)])
        efp = len(stat[(stat['ensem_preds'] == 1) & (stat['test_labels'] == 0)])
        
        print('R TP:{:>3} FP:{:>3}'.format(rtp, rfp))
        print('M TP:{:>3} FP:{:>3}'.format(mtp, mfp))
        print('E TP:{:>3} FP:{:>3}'.format(etp, efp))
        print('G TP:{:>+3} FP:{:>+3}'.format(etp - mtp, efp - mfp))
        print('-' * 50)

        mp, mr, mf, _ = precision_recall_fscore_support(
            stat.test_labels.tolist(), stat.test_preds.tolist(), average='binary')
        rp, rr, rf, _ = precision_recall_fscore_support(
            stat.test_labels.tolist(), stat.rule_preds.tolist(), average='binary')
        ep, er, ef, _ = precision_recall_fscore_support(
            stat.test_labels.tolist(), stat.ensem_preds.tolist(), average='binary')
        print('R P: {:.4f} R: {:.4f} F: {:.4f}'.format(rp, rr, rf))
        print('M P: {:.4f} R: {:.4f} F: {:.4f}'.format(mp, mr, mf))
        print('E P: {:.4f} R: {:.4f} F: {:.4f}'.format(ep, er, ef))
        print('G P:{:+.4f} R:{:+.4f} F:{:+.4f}'.format(ep - mp, er - mr, ef - mf))

        print('*' * 50)
        fp_sample = stat[stat['sentence'].isin(
            stat[(stat['test_preds'] == 0) & (stat['test_labels'] == 0) &
                 (stat['rule_preds'] == 1) &
                 (stat['ensem_preds'] == 1)]['sentence'])]
        fp_sample['time'] = time
        if fp_samples is None:
            fp_samples = fp_sample
        else:
            fp_samples = fp_samples.append(fp_sample)
        tpfp.append([etp-mtp, efp-mfp])
        all_gain.append([ep - mp, er - mr, ef - mf])
    return tpfp, all_gain, fp_samples

In [192]:
p = 0.7  # 规则置信度
r = 0.5  # 调和系数，调和模型和规则的概率
eval_root = '/data/wujipeng/ec/eval/'
model = 'han_v3_noft_wiki'

In [196]:
tpfps = []
all_gains = []
fp_samples = []
ps = np.arange(0.5, 1.05, 0.05)
for p in ps:
    tpfp, all_gain, fp_sample = stat_rule(model, p)
    tpfps.append(np.mean(tpfp, axis=0))
    all_gains.append(np.mean(all_gain, axis=0))
    fp_samples.append(fp_sample)

Eval time 1
R TP: 30 FP:  7
M TP:160 FP: 46
E TP:160 FP: 46
G TP: +0 FP: +0
--------------------------------------------------
R P: 0.8108 R: 0.1389 F: 0.2372
M P: 0.7767 R: 0.7407 F: 0.7583
E P: 0.7767 R: 0.7407 F: 0.7583
G P:+0.0000 R:+0.0000 F:+0.0000
**************************************************
Eval time 2
R TP: 38 FP:  4
M TP:149 FP: 45
E TP:149 FP: 45
G TP: +0 FP: +0
--------------------------------------------------
R P: 0.9048 R: 0.1767 F: 0.2957
M P: 0.7680 R: 0.6930 F: 0.7286
E P: 0.7680 R: 0.6930 F: 0.7286
G P:+0.0000 R:+0.0000 F:+0.0000
**************************************************
Eval time 3
R TP: 29 FP:  3
M TP:141 FP: 45
E TP:141 FP: 45
G TP: +0 FP: +0
--------------------------------------------------
R P: 0.9062 R: 0.1330 F: 0.2320
M P: 0.7581 R: 0.6468 F: 0.6980
E P: 0.7581 R: 0.6468 F: 0.6980
G P:+0.0000 R:+0.0000 F:+0.0000
**************************************************
Eval time 4
R TP: 23 FP:  4
M TP:150 FP: 52
E TP:150 FP: 52
G TP: +0 FP: +0
------

/home/wujipeng/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



E P: 0.7030 R: 0.6574 F: 0.6794
G P:-0.0035 R:+0.0000 F:-0.0016
**************************************************
Eval time 1
R TP: 30 FP:  7
M TP:160 FP: 46
E TP:160 FP: 46
G TP: +0 FP: +0
--------------------------------------------------
R P: 0.8108 R: 0.1389 F: 0.2372
M P: 0.7767 R: 0.7407 F: 0.7583
E P: 0.7767 R: 0.7407 F: 0.7583
G P:+0.0000 R:+0.0000 F:+0.0000
**************************************************
Eval time 2
R TP: 38 FP:  4
M TP:149 FP: 45
E TP:150 FP: 45
G TP: +1 FP: +0
--------------------------------------------------
R P: 0.9048 R: 0.1767 F: 0.2957
M P: 0.7680 R: 0.6930 F: 0.7286
E P: 0.7692 R: 0.6977 F: 0.7317
G P:+0.0012 R:+0.0047 F:+0.0031
**************************************************
Eval time 3
R TP: 29 FP:  3
M TP:141 FP: 45
E TP:141 FP: 45
G TP: +0 FP: +0
--------------------------------------------------
R P: 0.9062 R: 0.1330 F: 0.2320
M P: 0.7581 R: 0.6468 F: 0.6980
E P: 0.7581 R: 0.6468 F: 0.6980
G P:+0.0000 R:+0.0000 F:+0.0000
*****************

In [195]:
print('{:^4} {:^4} {:^4} {:^9} {:^9} {:^9}'.format('p', 'TP', 'FP', 'P', 'R', 'F1'))
for p, tpfp, all_gain in zip(ps, tpfps, all_gains):
    print('{:.2f} {:4.0f} {:4.0f} {:+.6f} {:+.6f} {:+.6f}'.format(p, *tpfp * 25, *all_gain))

 p    TP   FP      P         R        F1    
0.50    0    0 +0.000000 +0.000000 +0.000000
0.55    6    1 +0.000137 +0.001111 +0.000688
0.60   13    5 -0.000216 +0.002398 +0.001276
0.65   20    8 -0.000369 +0.003696 +0.001955
0.70   25   12 -0.000689 +0.004616 +0.002307
0.75   34   18 -0.001204 +0.006276 +0.002992
0.80   41   22 -0.001436 +0.007566 +0.003564
0.85   49   27 -0.001794 +0.009050 +0.004212
0.90   55   37 -0.003000 +0.010157 +0.004257
0.95   69   46 -0.003729 +0.012739 +0.005382
1.00   86   78 -0.007887 +0.015865 +0.005206


In [197]:
fp_sample = fp_samples[-1]
fp_sample = fp_sample.sort_values(by=['sentence', 'time'])
fp_sample.to_csv('/data/wujipeng/ec/eval/han_v3_noft_wiki/fp_samples.csv', decimal='.', sep=' ', index=False, float_format='%.4f')

In [10]:
pos0 = 69
row = raw_data.loc[402]
clauses = [clause.strip().split(' ') for clause in row['clause'].strip().split('\x01')]
natures = [nature.strip().split(' ') for nature in row['clause'].strip().split('\x01')]
keyword = row['keyword']
f = [int(pos) for pos in row['clause_pos'].strip().split(' ')].index(pos0)
found = rule3(f, clauses, natures, keyword, strict=False)